<a href="https://colab.research.google.com/github/sinajahangir/FloodAgent/blob/main/FloodAgentDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This AI agent code is designed to extract location parameters (coordinates) from a user prompt using natural language processing. After identifying the relevant location, the agent queries a flood information system to retrieve and present current flood-related details (e.g., severity, vulnerability) specific to the extracted location.
The case study presented in this notebook is associated with City of Calgary

You do not need to run the notebook on GPU

Developed by: Sina Jahangir (first version: October 2025)
license: GNU GENERAL PUBLIC LICENSE

# Clone repository

After executing, you should see the repo in Files section

In [1]:
import os
if os.path.isdir('FloodAgent'):
    !rm -rf FloodAgent #removed if already there is a cloned repo
!git clone https://github.com/sinajahangir/FloodAgent.git

Cloning into 'FloodAgent'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 55 (delta 28), reused 36 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (55/55), 3.54 MiB | 8.62 MiB/s, done.
Resolving deltas: 100% (28/28), done.


# Install libraries and load dependencies

In [2]:
# Execute the setup.py file that is now in the cloned directory
%run FloodAgent/SetupLibraries.py

Package ''pgmpy'' not found. Attempting installation of all required packages...
Success: Packages installed and imported after retry.
Numpy version: 2.0.2
Setup script finished execution


# LLM setup

In [3]:
# --- LLM Setup ---
try:
    # Using Colab Secrets for API key management
    api_key = userdata.get('RAG') #API key needs setup in the cloud. "RAG" is an arbitrary name example
    genai.configure(api_key=api_key)
    print("Gemini API Key configured.")
    # Initialize the generative model
    llm_model = genai.GenerativeModel('gemini-2.5-flash-lite') # Use a model good at instruction following/extraction
    print(f"LLM Model '{llm_model.model_name}' initialized.")
except userdata.SecretNotFoundError:
    print("ERROR: Gemini API Key ('GEMINI_API_KEY') not found in Colab Secrets.")
    print("Please add your API key via the 'Secrets' tab (key icon) on the left.")
    llm_model = None
except Exception as e:
    print(f"An error occurred during Gemini setup: {e}")
    llm_model = None

Gemini API Key configured.
LLM Model 'models/gemini-2.5-flash-lite' initialized.


# Agent class

In [4]:
from FloodAgent.AgentSource import CoordinateFloodProximityAgent

# Read data

In [5]:
df_flood=pd.read_csv('FloodAgent/Data/CA_202012_FLSW_U_RP100_RB_30m_4326.csv') #flood data
df_sovi=pd.read_csv('FloodAgent/Data/DA_SoVI_Coordinates.csv') #Social Vulnerability
modeli_=DiscreteBayesianNetwork.load('FloodAgent/Data/BBN_Model_RoadNetwork_Updated.bif', filetype='bif') #Bayesian network

In [6]:
#Convert dataframes to numpy array (expected by the agent)
flood_pixels = np.array(df_flood[['easting', 'northing']])
sovi_pixels = np.array(df_sovi[['easting', 'northing']])
sovi_array = np.array(df_sovi['SoVI'])
pop_density_array = np.array(df_sovi['pop_d_c'])

#Test Agent

In [7]:
# --- Run Example IF LLM is available and data is ready ---
if llm_model and len(flood_pixels) > 0:
    # Instantiate the agent
    try:
        agent = CoordinateFloodProximityAgent(
            flood_pixel_coords=flood_pixels,# Essential: Flood pixel data
            sovi_coords=sovi_pixels,# SoVI coordination
            sovi_array=sovi_array, # SoVI data
            pop_density_array=pop_density_array,
            BBN=modeli_, #Bayesian network,
            llm_model=llm_model, #LLM model
            show_map=True #Show map
        )

        prompt1 = "Analyze flood at the largest hospital in Calgary, Foothills Medical Centre, located at coordinates 51.065389°N, 114.133306°W?"
        print(f"\nUser Query 1: {prompt1}")
        response1 = agent.find_closest_flood_pixel_to_location(prompt1)
        print(f"Agent Response 1:\n{response1}")

    except Exception as e:
        print(f"\nAn error occurred during agent instantiation or testing: {e}")

elif not llm_model:
     print("\nSkipping example usage: LLM model not initialized.")
else:
     print("\nSkipping example usage: Placeholder data (buildings or flood pixels) is empty or missing.")

Stored 191478 flood pixel coordinates.
Building k-d tree for flood pixels...
Flood k-d tree built.

User Query 1: Analyze flood at the largest hospital in Calgary, Foothills Medical Centre, located at coordinates 51.065389°N, 114.133306°W?
Displaying Location: Lat=51.065389, Lon=-114.133306
Direct Link (Open in New Tab): https://www.google.com/maps/@51.065389,-114.133306,10z


Converted to UTM (EPSG:32611): Easting=700854.339158512, Northing=5661006.428564519
Closest flood pixel found: Coords=[ 700936.80600551 5661075.03498424], Distance=107.2736
Agent Response 1:
Here's a concise response incorporating the provided information, with a modified tone based on the SoVI value class:

**"We found the closest flood record about 107 meters away from your location. This area has a 'Low' SoVI value, indicating a low level of social vulnerability. The population density is also low, and the expected road class is 'Local'. Considering these factors, your flood exposure is rated as 'Moderate', and your flood resilience is estimated to be **Good**.**

***

**Chain of Thought for Flood Resilience Estimation:**

My estimation of "Good" flood resilience is based on the interplay of the provided data points, particularly focusing on the indicators of vulnerability and the characteristics of the flood event. Here's how I arrived at that conclusion:

1.  **SoVI Value Class (L